## Find variable objects in Rubin

This notebook finds the closest matches for our 10 variable objects in Rubin.

In [ ]:
import lsdb
import pandas as pd

from nested_pandas import NestedDtype
from pathlib import Path

In [ ]:
# We add the index as a column of the DataFrame so we can later retrieve
# all the Rubin data from difference and science imaging
variables_df = pd.read_csv("periodic_objects.csv").reset_index()
# Transform the DataFrame into a LSDB Catalog
variables_catalog = lsdb.from_dataframe(variables_df)

In [ ]:
drp_release = "w_2025_10"
base_dir = Path("/sdf/data/rubin/shared/lsdb_commissioning/hats")
hats_dir = base_dir / drp_release

In [ ]:
object_lc_x_ps1 = lsdb.read_hats(hats_dir / "object_lc_x_ps1")
object_lc_x_ps1

In [ ]:
def cast_nested(df, columns):
    return df.assign(
        **{
            col: df[col].astype(NestedDtype.from_pandas_arrow_dtype(df.dtypes[col]))
            for col in columns
        },
    )

In [ ]:
object_lc = lsdb.read_hats(
    hats_dir / "object_lc",
    margin_cache=hats_dir / "object_lc_5arcs",
    columns=["objectId", "coord_ra", "coord_dec", "forcedSource"],
)
# We use the `cast_nested` utility method to cast columns into the NestedFrame type
object_lc = object_lc.map_partitions(cast_nested, columns=["forcedSource"])

In [ ]:
# Get the Rubin data for our objects
object_lc = variables_catalog.crossmatch(object_lc, radius_arcsec=0.2, suffixes=["", ""])

In [ ]:
# Save all calculated Rubin periods to disk
object_lc.to_hats("rubin_variables", catalog_name="rubin_variables")